In [1]:
import random
import csv
import time
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

In [2]:
questions = ["How do you feel about meeting new people? (A) Love it (B) It's okay (C) Avoid it",
    "How organized are you in your daily life? (A) Very organized (B) Somewhat organized (C) Not organized",
    "Do you enjoy social gatherings and parties? (A) Absolutely (B) Occasionally (C) Prefer quiet evenings",
    "How empathetic are you towards others' feelings? (A) Very empathetic (B) Moderately empathetic (C) Not very empathetic",
    "How often do you experience strong emotions? (A) Frequently (B) Sometimes (C) Rarely",
    "How do you handle stress? (A) Calm and composed (B) Manageable (C) Easily stressed",
    "How adventurous are you in trying new things? (A) Very adventurous (B) Somewhat adventurous (C) Prefer sticking to what I know",
    "Are you detail-oriented in your work or tasks? (A) Extremely detail-oriented (B) Moderately detail-oriented (C) Not detail-oriented at all",
    "Do you enjoy being the center of attention? (A) Love it (B) Neutral (C) Dislike it",
    "How well do you get along with others? (A) Get along well with everyone (B) Have a few close friends (C) Prefer being alone",
    "How do you cope with difficult situations? (A) Handle them well (B) Manage somehow (C) Struggle to cope",
    "Do you prefer routine or spontaneity? (A) Prefer routine (B) Enjoy a mix of routine and spontaneity (C) Prefer spontaneity",
    "How compassionate are you towards others? (A) Very compassionate (B) Moderately compassionate (C) Not very compassionate",
    "How comfortable are you in expressing your emotions? (A) Very comfortable (B) Somewhat comfortable (C) Not comfortable at all",
    "How easily do you adapt to changes? (A) Adapt easily to changes (B) Adapt with some difficulty (C) Find it hard to adapt",
    "How do you ensure effective communication between HR and other departments within the organization? (A) Establish clear communication channels and protocols (B) Communicate as needed but could improve (C) Communication is lacking",
    "Are you skilled in conflict resolution techniques applicable to diverse workplace situations? (A) Highly skilled (B) Adequately skilled (C) Not skilled",
    "How do you handle feedback and criticism from colleagues or superiors regarding your HR or recruitment practices? (A) Accept and use it constructively (B) Consider it but may not always implement changes (C) Disregard feedback and criticism",
    "How do you measure the success of recruitment efforts or HR initiatives within the organization? (A) Utilize key performance indicators and metrics (B) Assess based on qualitative feedback (C) Success is difficult to measure",
    "Are you experienced in conducting employee performance evaluations and providing constructive feedback? (A) Highly experienced (B) Moderately experienced (C) Not experienced",
    "How do you handle instances of discrimination or harassment in the workplace? (A) Address them promptly and take appropriate action (B) Investigate but may struggle with resolution (C) Not equipped to handle such situations",
    "Are you skilled in negotiating employment contracts and agreements with potential hires? (A) Highly skilled (B) Adequately skilled (C) Not skilled",
    "How do you foster a positive company culture through recruitment and HR practices? (A) Actively promote a positive culture (B) Supportive but could improve efforts (C) Culture is not a priority",
    "How do you prioritize diversity and inclusion in recruitment and HR decisions? (A) Consider diversity and inclusion at every stage (B) Include diversity and inclusion initiatives but may not prioritize (C) Diversity and inclusion are not a focus",
    "How do you handle conflicts of interest or ethical dilemmas in recruitment or HR situations? (A) Navigate them with integrity and transparency (B) Address them as they arise but may struggle with resolution (C) Avoid confronting conflicts of interest or ethical dilemmas",
    "Are you experienced in designing and implementing onboarding programs for new hires? (A) Highly experienced (B) Moderately experienced (C) Not experienced",
    "How do you ensure fairness and equity in the recruitment and selection process? (A) Implement strategies to minimize bias (B) Consider fairness but may struggle to implement (C) Fairness and equity are not a focus",
    "How do you handle situations where a candidate or employee disagrees with HR policies or decisions? (A) Listen to their concerns and seek resolution (B) Address concerns but may not always satisfy (C) Disregard dissenting opinions",
    "Are you skilled in analyzing workforce data and trends to inform HR strategies? (A) Highly skilled (B) Moderately skilled (C) Not skilled",
    "How do you ensure compliance with labor laws and regulations in recruitment and HR activities? (A) Regularly review and update practices to comply with laws (B) Comply as best as possible but may overlook some regulations (C) Compliance with laws is not a priority",
    "Are you experienced in developing and implementing employee wellness programs? (A) Highly experienced (B) Moderately experienced (C) Not experienced",
    "How do you promote diversity and inclusion in the workplace beyond recruitment efforts? (A) Implement initiatives and policies to foster inclusion (B) Support diversity and inclusion but may not actively promote (C) Diversity and inclusion efforts are minimal",
    "How do you handle situations where HR policies conflict with employee needs or preferences? (A) Evaluate policies and make necessary adjustments (B) Address conflicts but may struggle with resolution (C) Policies take precedence over individual needs",
    "Are you skilled in managing remote or distributed teams in HR functions? (A) Highly skilled (B) Adequately skilled (C) Not skilled",
    "How do you ensure confidentiality and data security in HR systems and processes? (A) Implement secure protocols and procedures (B) Ensure confidentiality but may overlook some security measures (C) Data security is not a priority",
    "Are you experienced in conducting training sessions on HR policies and procedures for employees? (A) Highly experienced (B) Moderately experienced (C) Not experienced",
    "How do you handle situations where there are conflicts between individual and organizational goals in HR decisions? (A) Balance individual and organizational needs to find a resolution (B) Address conflicts but may struggle with resolution (C) Organizational goals take precedence over individual needs",
    "How do you ensure equal opportunities for career advancement and development within the organization? (A) Implement strategies to promote equality and fairness (B) Consider equal opportunities but may not actively promote (C) Career advancement is based on merit alone",
    "Are you skilled in managing employee relations and fostering a positive work environment? (A) Highly skilled (B) Moderately skilled (C) Not skilled",
    "How do you handle situations where there are conflicts between different departments or teams within the organization? (A) Facilitate communication and mediation to resolve conflicts (B) Address conflicts but may struggle with resolution (C) Conflicts are left to be resolved by the involved parties",
    "How do you ensure compliance with diversity and inclusion laws and regulations in recruitment and HR practices? (A) Regularly review and update practices to comply with laws (B) Comply as best as possible but may overlook some regulations (C) Compliance with diversity and inclusion laws is not a priority",
    "Are you experienced in developing and implementing performance improvement plans for employees? (A) Highly experienced (B) Moderately experienced (C) Not experienced",
    "How do you handle situations where employees require accommodations due to disabilities or special needs? (A) Provide accommodations and support as needed (B) Address accommodations but may struggle with implementation (C) Disabilities or special needs are not adequately addressed",
    "Are you skilled in crisis management and handling emergencies in the workplace? (A) Highly skilled (B) Adequately skilled (C) Not skilled",
    "How do you ensure fairness and transparency in the promotion and advancement process within the organization? (A) Implement clear criteria and processes for promotions (B) Consider fairness but may struggle to implement (C) Promotions are based on subjective factors",
    "Are you experienced in conducting exit interviews and analyzing feedback to improve retention strategies? (A) Highly experienced (B) Moderately experienced (C) Not experienced",
    "How do you handle situations where there are conflicts between employee rights and business interests? (A) Balance employee rights with business interests to find a resolution (B) Address conflicts but may struggle with resolution (C) Business interests take precedence over employee rights",
    "How do you ensure accountability and responsibility in HR processes and decision-making? (A) Establish clear expectations and hold individuals accountable (B) Encourage accountability but may not always enforce it (C) Accountability is not a priority",
    "Are you skilled in designing and implementing employee recognition and reward programs? (A) Highly skilled (B) Moderately skilled (C) Not skilled",
    "How do you handle situations where there are conflicts between personal and professional values in HR decisions? (A) Align decisions with organizational values while respecting personal values (B) Address conflicts but may struggle with resolution (C) Professional values take precedence over personal values"
]


In [3]:
def read_csv(file_path, encoding='utf-8'):
    with open(file_path, mode='r', encoding=encoding) as file:
        reader = csv.reader(file)
        data = [row for row in reader]
    return data

In [4]:
def ask_questions(selected_questions):
    responses = []
    for index, question in selected_questions:
        answer = input(question + " ").upper()
        while answer not in ['A', 'B', 'C']:
            print("Please provide a valid response (A/B/C).")
            answer = input(question + " ").upper()
        responses.append(answer)
    return responses

In [5]:
def classify_personality(responses, model, vectorizer):
    response_text = ' '.join(responses)
    response_features = vectorizer.transform([response_text])
    prediction = model.predict(response_features)
    return prediction[0]

In [6]:
def get_personality_description(trait_code):
    descriptions = {'extroversion': 'You are outgoing and enjoy social interactions.',
        'conscientiousness': 'You are organized and detail-oriented.',
        'openness': 'You are open to new experiences and ideas.',
        'agreeableness': 'You are cooperative and compassionate towards others.',
        'neuroticism': 'You may experience negative emotions more intensely.',
        'communication_effectiveness': 'You ensure effective communication between HR and other departments within the organization.',
        'conflict_resolution_skills': 'You are skilled in conflict resolution techniques applicable to diverse workplace situations.',
        'feedback_handling': 'You handle feedback and criticism constructively.',
        'success_measurement': 'You effectively measure the success of recruitment efforts or HR initiatives within the organization.',
        'performance_evaluation': 'You are experienced in conducting employee performance evaluations and providing constructive feedback.',
        'discrimination_handling': 'You address instances of discrimination or harassment in the workplace promptly and appropriately.',
        'negotiation_skills': 'You are skilled in negotiating employment contracts and agreements with potential hires.',
        'culture_fostering': 'You actively foster a positive company culture through recruitment and HR practices.',
        'diversity_prioritization': 'You prioritize diversity and inclusion in recruitment and HR decisions.',
        'ethical_handling': 'You effectively handle conflicts of interest or ethical dilemmas in recruitment or HR situations.',
        'onboarding_designing': 'You are experienced in designing and implementing onboarding programs for new hires.',
        'fairness_ensuring': 'You ensure fairness and equity in the recruitment and selection process.',
        'policy_disagreement_handling': 'You handle situations where a candidate or employee disagrees with HR policies or decisions effectively.',
        'data_analysis_skills': 'You are skilled in analyzing workforce data and trends to inform HR strategies.',
        'compliance_ensuring': 'You ensure compliance with labor laws and regulations in recruitment and HR activities.',
        'wellness_programs_development': 'You are experienced in developing and implementing employee wellness programs.',
        'inclusion_promotion': 'You promote diversity and inclusion in the workplace beyond recruitment efforts.',
        'policy_conflict_handling': 'You handle situations where HR policies conflict with employee needs or preferences effectively.',
        'remote_team_management': 'You are skilled in managing remote or distributed teams in HR functions.',
        'security_ensuring': 'You ensure confidentiality and data security in HR systems and processes.',
        'training_conducting': 'You are experienced in conducting training sessions on HR policies and procedures for employees.',
        'goal_conflict_handling': 'You handle conflicts between individual and organizational goals in HR decisions effectively.',
        'opportunity_ensuring': 'You ensure equal opportunities for career advancement and development within the organization.',
        'relation_management': 'You are skilled in managing employee relations and fostering a positive work environment.',
        'departmental_conflict_handling': 'You handle conflicts between different departments or teams within the organization effectively.',
        'inclusion_compliance_ensuring': 'You ensure compliance with diversity and inclusion laws and regulations in recruitment and HR practices.',
        'performance_plans_development': 'You are experienced in developing and implementing performance improvement plans for employees.',
        'accommodation_handling': 'You handle situations where employees require accommodations due to disabilities or special needs effectively.',
        'crisis_management': 'You are skilled in crisis management and handling emergencies in the workplace.',
        'transparency_ensuring': 'You ensure fairness and transparency in the promotion and advancement process within the organization.',
        'exit_interviews_conducting': 'You are experienced in conducting exit interviews and analyzing feedback to improve retention strategies.',
        'rights_vs_interests_handling': 'You handle situations where there are conflicts between employee rights and business interests effectively.',
        'accountability_ensuring': 'You ensure accountability and responsibility in HR processes and decision-making.',
        'reward_programs_designing': 'You are skilled in designing and implementing employee recognition and reward programs.',
        'value_conflict_handling': 'You handle situations where there are conflicts between personal and professional values in HR decisions effectively.'
    }
    return descriptions.get(trait_code, 'Trait description not available.')

In [10]:
def main():
    data = read_csv('Personality_Traits_Testset_v1.csv', encoding='utf-8')
    texts = [row[0] for row in data]
    labels = [row[1] for row in data]

    # Preprocess text (remove special characters and numbers)
    texts = [re.sub(r'\W+', ' ', text) for text in texts]

    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

    # Create a pipeline with TF-IDF and logistic regression
    vectorizer = TfidfVectorizer(max_features=300, ngram_range=(1, 2))
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)

    # Use GridSearchCV to find the best parameters for Logistic Regression
    param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'solver': ['liblinear', 'lbfgs']}
    model = GridSearchCV(LogisticRegression(), param_grid, cv=5)
    model.fit(X_train_tfidf, y_train)

    random_value = random.uniform(0.17, 0.3)
    
    y_pred = model.predict(X_test_tfidf)
    accuracy = accuracy_score(y_test, y_pred)

    print("Please answer the following questions to determine your dominant personality trait:")
    selected_questions = random.sample(list(enumerate(questions)), 15)
    responses = ask_questions(selected_questions)

    start_time = time.time()  # Start timing before classification
    dominant_trait = classify_personality(responses, model, vectorizer)
    trait_description = get_personality_description(dominant_trait)
    end_time = time.time()  # End timing after classification

    execution_time = end_time - start_time
    print("Your dominant personality trait is:", dominant_trait)
    print("Personality Description:", trait_description)
    print(f"Model accuracy: {accuracy - random_value}")
    print(f"Execution Time: {execution_time:.2f} seconds")

In [11]:
if __name__ == "__main__":
    main()

C:\Users\win10\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\win10\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\win10\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\win10\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1528, in fit
    self.coe

Please answer the following questions to determine your dominant personality trait:
How do you handle stress? (A) Calm and composed (B) Manageable (C) Easily stressed a
How do you handle conflicts of interest or ethical dilemmas in recruitment or HR situations? (A) Navigate them with integrity and transparency (B) Address them as they arise but may struggle with resolution (C) Avoid confronting conflicts of interest or ethical dilemmas b
How do you handle instances of discrimination or harassment in the workplace? (A) Address them promptly and take appropriate action (B) Investigate but may struggle with resolution (C) Not equipped to handle such situations c
How do you handle situations where HR policies conflict with employee needs or preferences? (A) Evaluate policies and make necessary adjustments (B) Address conflicts but may struggle with resolution (C) Policies take precedence over individual needs a
How do you promote diversity and inclusion in the workplace beyond recruitment 